In [ ]:
build = 'archive' # 'archive' uses the neurolibre archive of the data., 'latest' would download the latest versions of the data.
notebook = 'figures'

<head>

<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=STIX+Two+Text:ital,wght@0,400;0,700;1,400&display=swap" rel="stylesheet">
</head>


<p>
<center>
<b>
<h3>
Analysis code for the paper "RF shimming in the cervical spinal cord at 7T"
</h3>
</b>
</center>


# 1 &nbsp; &nbsp; | &nbsp; &nbsp; Data

The data can be downloaded from: https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):

```{toggle}
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~
```

Test citation: {cite:p}`DAstous2023`.

# 2 &nbsp; &nbsp; | &nbsp; &nbsp; Overview of processing pipeline

For each subject:

- Process anat/T2starw (GRE)
  - Segment the spinal cord (SC)
  - Label vertebral levels using existing manual disc labels
  - Create a mask of the cerebrospinal fluid (CSF)
  - Extract the SC/CSF magnitude signal to assess the stability of the flip angle across shim methods
- Process fmap/TFL (flip angle maps)
  - Register each B1 map (CP, CoV, etc.) to the GRE scan
  - Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
  - Convert the B1 map to nT/V units
  - Extract the B1 map value within the SC

>Slow processes are indicated with the emoji ⏳

# 3 &nbsp; &nbsp; | &nbsp; &nbsp; Requirements

* Install [Spinal Cord Toolbox](https://spinalcordtoolbox.com/user_section/installation.html), eg

```shell
# Install Python libaries
!wget -O requirements.txt https://raw.githubusercontent.com/shimming-toolbox/rf-shimming-7t/main/requirements.txt
!pip install -r requirements.txt

# Install SCT ⏳
!git clone --depth 1 https://github.com/spinalcordtoolbox/spinalcordtoolbox.git
!yes | spinalcordtoolbox/install_sct
os.environ['PATH'] += f":/content/spinalcordtoolbox/bin"
```

* Download the project repository

```shell
git clone https://github.com/shimming-toolbox/rf-shimming-7t-neurolibre.git
cd rf-shimming-7t-neurolibre
```

* Install requirements

```shell
pip install -e binder/requirements.txt
```

* Download data

```shell
cd content
repo2data -r ../binder/data_requirement.json
```


# 3 &nbsp; &nbsp; | &nbsp; &nbsp; Environment setup

In a Python shell:

Import the necessary modules.

In [ ]:
from datetime import datetime, timedelta
import os
import re
import json
import subprocess
import glob
import itertools
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from scipy.stats import f_oneway, ttest_rel
import shutil
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm

Define variables.

In [ ]:
os.chdir("../data/rf-shimming-7t/ds004906")

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_labels = os.path.join(path_data, "derivatives", "labels")
path_qc = os.path.join(path_data, "qc")
shim_modes = ["CP", "patient", "volume", "phase", "CoV", "target", "SAReff"]
shim_modes_MPRAGE = ["CP", "CoV"] 

print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, 'derivatives', 'results')
os.makedirs(path_results, exist_ok=True)

# References 

```{bibliography}
:filter: docname in docnames
```